# Fine Tuning RoBERTa

link: https://towardsdatascience.com/news-category-classification-fine-tuning-roberta-on-tpus-with-tensorflow-f057c37b093


In [8]:
import numpy as np
import sklearn
from sklearn import metrics
import transformers
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import json
import matplotlib.pyplot as plt
import random
import seaborn as sn
import torch

In [5]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
"""
When running in Colab:

# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

batch_size=32 * tpu_strategy.num_replicas_in_sync
print('Batch size:', batch_size)
AUTOTUNE = tf.data.experimental.AUTOTUNE
"""

In [2]:
#Load data from json file
data = [json.loads(line) for line in open('../../data/News_Category_Dataset_v3.json', 'r')] 
random.shuffle(data) #shuffle the data 
labels=[]
headlines=[]
texts=[]
for line in data:
    labels.append(line['category'])
    headlines.append(line['headline'])
    #Combine headline and description into a single text input
    text=line['headline']+' '+line['short_description']
    texts.append(text)

In [27]:
def aggregate_categories(labels):
    aggregated=[]
    for line in labels:
        if line=='WORLDPOST' or line=='THE WORLDPOST':
            aggregated.append('WORLD NEWS')
        elif line=='PARENTING' or line=='PARENTS':
            aggregated.append('PARENTS & PARENTING')
        elif line=='ARTS' or line=='CULTURE & ARTS':
            aggregated.append('ARTS & CULTURE')
        elif line=='STYLE':
            aggregated.append('STYLE & BEAUTY')
        elif line=='GREEN' or line=='ENVIRONMENT':
            aggregated.append('ENVIRONMENT & GREEN')
        elif line=='HEALTHY LIVING' or line=='WELLNESS':
            aggregated.append('WELLNESS & HEALTHY LIVING')
        elif line=='COMEDY' or line=='ENTERTAINMENT':
            aggregated.append('ENTERTAINMENT & COMEDY')
        elif line=='TASTE' or line=='FOOD & DRINK':
            aggregated.append('FOOD, DRINK & TASTE')
        elif line=='COLLEGE' or line=='EDUCATION':
             aggregated.append('COLLEGE & EDUCATION')
        elif line=='SCIENCE' or line=='TECH':
             aggregated.append('SCIENCE & TECH')
        elif line=='BUSINESS' or line=='MONEY':
            aggregated.append('BUSINESS & MONEY')

        else:
            aggregated.append(line)
    return aggregated

aggregated=aggregate_categories(labels)

In [28]:
categories=sorted(list(set(labels))) #set will return the unique different entries
n_categories=len(categories)

def indicize_labels(labels):
    """Transforms string labels into indices"""
    indices=[]
    for j in range(len(labels)):
        for i in range(n_categories):
            if labels[j]==categories[i]:
                indices.append(i)
    return indices

indices=indicize_labels(aggregated) #Integer label indices


In [ ]:
"""
# For Tensor Flow
tokenizer = AutoTokenizer.from_pretrained("roberta-base") #Tokenizer
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='tf') #Tokenized text
"""

In [24]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base") #Tokenizer
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt') #Tokenized text for pytorch
# docs: https://huggingface.co/docs/transformers/preprocessing

type(inputs)

transformers.tokenization_utils_base.BatchEncoding

In [29]:
dict(inputs)

{'input_ids': tensor([[    0, 16215, 19716,  ...,     1,     1,     1],
         [    0, 34985,   876,  ...,     1,     1,     1],
         [    0,   108,   574,  ...,     1,     1,     1],
         ...,
         [    0,  6715,  5066,  ...,     1,     1,     1],
         [    0, 11773,    17,  ...,     1,     1,     1],
         [    0,  9690, 28514,  ...,     1,     1,     1]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])}

In [32]:
#dataset=tf.data.Dataset.from_tensor_slices((dict(inputs), indices)) #Create a pytorch dataset

dataset=torch.utils.data.TensorDataset(inputs, indices) #Create a pytorch dataset


AttributeError: 

In [21]:
n_elements=len(headlines)


#train test split, we use 10% of the data for validation
val_data_size=int(0.1*n_elements)
val_ds=dataset.take(val_data_size).batch(batch_size, drop_remainder=True) 
train_ds=dataset.skip(val_data_size).batch(batch_size, drop_remainder=True)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

NameError: name 'dataset' is not defined